In [1]:
# Carga de librerias
from IPython.display import display
import subprocess as sp
import numpy as np
import pandas as pd
import os
import glob
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import rcParams
from sklearn.metrics import r2_score
from statsmodels.tsa.seasonal import seasonal_decompose
from pylab import rcParams
import random
import time

# Configurar parametros de pandas y matplotlib
pd.set_option('display.precision', 4)
%config IPCompleter.greedy=True
%matplotlib inline

params = {}
plt.rcParams.update(params)
params = {
           'legend.fontsize': 20,
           'legend.handlelength': 2,
           'axes.titlesize':'20',
           'xtick.labelsize':'20',
           'ytick.labelsize':'20'
         }

matplotlib.rcParams.update(params)

In [4]:
import os
import pandas as pd

def unificar_archivos_excel(carpeta_principal, archivo_salida):
    # Lista para almacenar todos los dataframes
    dfs = []
    
    # Recorre todos los archivos en la carpeta principal y subcarpetas
    for root, dirs, files in os.walk(carpeta_principal):
        for file in files:
            if file.endswith('.csv') or file.endswith('.xlsx'):  # Verifica si es un archivo Excel
                archivo_actual = os.path.join(root, file)
                try:
                    # Lee el archivo Excel y agrega su contenido a la lista
                    df = pd.read_csv(archivo_actual)
                    dfs.append(df)
                except Exception as e:
                    print(f"No se pudo leer el archivo {archivo_actual}: {e}")

    # Si hay archivos leídos, unificarlos en un solo DataFrame
    if dfs:
        df_unificado = pd.concat(dfs, ignore_index=True)
        
        # Guardar el DataFrame unificado en un archivo Excel
        df_unificado.to_csv(archivo_salida, index=False)
        print(f"Archivos unificados en {archivo_salida}")
    else:
        print("No se encontraron archivos Excel.")

# Usar el script
carpeta_calidad = 'data/raw/calidad'  # Ruta a la carpeta principal
archivo_calidad = 'unificado_calidad.csv'  # Nombre del archivo Excel de salida
df_calidad = unificar_archivos_excel(carpeta_calidad, archivo_calidad)

carpeta_humedad = 'data/raw/humedad'  # Ruta a la carpeta principal
archivo_humedad = 'unificado_humedad.csv'  # Nombre del archivo Excel de salida
df_humedad = unificar_archivos_excel(carpeta_humedad, archivo_humedad)

carpeta_precipitacion = 'data/raw/precipitacion'  # Ruta a la carpeta principal
archivo_precipitacion = 'unificado_precipitacion.csv'  # Nombre del archivo Excel de salida
df_precipitacion = unificar_archivos_excel(carpeta_precipitacion, archivo_precipitacion)

carpeta_presion = 'data/raw/presion'  # Ruta a la carpeta principal
archivo_presion = 'unificado_presion.csv'  # Nombre del archivo Excel de salida
df_presion = unificar_archivos_excel(carpeta_presion, archivo_presion)

carpeta_temperatura = 'data/raw/temperatura'  # Ruta a la carpeta principal
archivo_temperatura = 'unificado_temperatura.csv'  # Nombre del archivo Excel de salida
df_temperatura = unificar_archivos_excel(carpeta_temperatura, archivo_temperatura)

carpeta_vientos = 'data/raw/vientos'  # Ruta a la carpeta principal
archivo_vientos = 'unificado_vientos.csv'  # Nombre del archivo Excel de salida
df_vientos = unificar_archivos_excel(carpeta_vientos, archivo_vientos)

Archivos unificados en unificado_calidad.csv
Archivos unificados en unificado_humedad.csv
Archivos unificados en unificado_precipitacion.csv
Archivos unificados en unificado_presion.csv
Archivos unificados en unificado_temperatura.csv
Archivos unificados en unificado_vientos.csv


In [ ]:
import os
import pandas as pd

# Ruta de la carpeta que contiene los archivos CSV
carpeta = "data/"

# Recorremos cada archivo en la carpeta
for archivo in os.listdir(carpeta):
    # Verificamos que el archivo termine en .csv y empiece con "unificado_"
    if archivo.endswith(".csv") and archivo.startswith("unificado_"):
        ruta_completa = os.path.join(carpeta, archivo)
        # Leemos el CSV
        df = pd.read_csv(ruta_completa)
        
        # Extraemos el sufijo del nombre: lo que sigue a "unificado_" y antes de ".csv"
        sufijo = archivo[len("unificado_"):-len(".csv")]
        
        # Si existe la columna "Calidad", la renombramos
        if "Calidad" in df.columns:
            nuevo_nombre = f"calidad_{sufijo}"
            df.rename(columns={"Calidad": nuevo_nombre}, inplace=True)
            print(f"Archivo: {archivo} - Se renombró 'Calidad' a '{nuevo_nombre}'")
        else:
            print(f"Archivo: {archivo} - No se encontró la columna 'calidad'")
        
        # Guardamos el DataFrame modificado sobrescribiendo el archivo original (o puedes guardarlo con otro nombre)
        df.to_csv(ruta_completa, index=False)   


Archivo: unificado_calidad.csv - No se encontró la columna 'calidad'
Archivo: unificado_humedad.csv - Se renombró 'Calidad' a 'calidad_humedad'
Archivo: unificado_precipitacion.csv - Se renombró 'Calidad' a 'calidad_precipitacion'
Archivo: unificado_presion.csv - Se renombró 'Calidad' a 'calidad_presion'
Archivo: unificado_temperatura.csv - Se renombró 'Calidad' a 'calidad_temperatura'
Archivo: unificado_vientos.csv - Se renombró 'Calidad' a 'calidad_vientos'


In [12]:
import os
import pandas as pd

# Ruta de la carpeta que contiene los archivos CSV
carpeta = "data/"

# Lista de archivos a unir
archivos = [
    "unificado_humedad.csv",
    "unificado_precipitacion.csv",
    "unificado_presion.csv",
    "unificado_temperatura.csv",
    "unificado_vientos.csv"
]

# Inicializamos el dataframe final
df_merged = None

# Iteramos sobre cada archivo
for archivo in archivos:
    ruta_completa = os.path.join(carpeta, archivo)
    # Leemos el archivo CSV
    df_temp = pd.read_csv(ruta_completa)
    
    # Si es el primer archivo, lo asignamos directamente
    if df_merged is None:
        df_merged = df_temp
    else:
        # Realizamos un merge usando "fecha_hora" como llave, utilizando inner para conservar los registros por minuto
        df_merged = pd.merge(df_merged, df_temp, on="fecha_hora", how="left")
    print(f"Archivo {archivo} unido correctamente.")

df_merged = df_merged.drop_duplicates(subset=["fecha_hora"])

# Guardamos el dataset combinado en un nuevo archivo CSV
ruta_salida = os.path.join(carpeta, "dataset_completo.csv")
df_merged.to_csv(ruta_salida, index=False)
print("Dataset combinado guardado en:", ruta_salida)


Archivo unificado_humedad.csv unido correctamente.
Archivo unificado_precipitacion.csv unido correctamente.
Archivo unificado_presion.csv unido correctamente.
Archivo unificado_temperatura.csv unido correctamente.
Archivo unificado_vientos.csv unido correctamente.
Dataset combinado guardado en: data/dataset_completo.csv
